## Real-Time Wildlife Alert System: Tutorial Notebook
Thomas Ratsakatika | AI and Environment Researcher | University of Cambridge

### Overview

xxx

### Step 1: Environment Set Up and Imports

- environment
- download models

In [ ]:
### Import required modules ###

from megadetector.detection import run_detector  # MegaDetector object detection model
import pandas as pd             # Data manipulation and analysis
import yaml                     # Handles YAML (config) files
import time                     # Time functions
from datetime import datetime   # Date and time manipulation
import schedule                 # Schedule function for weekly reports
import functools                # Function for weekly reports
import sys                      # System parameters and functions
sys.path.append('../scripts')   # Add scripts directory to system path

### Import modules required for the the example ###

from IPython.display import display, HTML  # Function to display images in Jupyter notebooks
import tkinter as tk            # Tkinter GUI toolkit
from tkinter import filedialog  # File dialog for opening files
from PIL import Image           # Python Imaging Library (PIL)
import os                       # Functions for interacting with the operating system
from io import BytesIO          # Handle binary data in memory
import base64                   # Encode binary data for HTML display

### Import utility functions from ../scripts/alert_system_utils.py ###

from alert_system_utils import (

    ### Functions to download photos and metadata from emails ###

    current_time,               # Get the current time
    check_emails,               # Checks for new emails, extracts photos and metadata
    extract_and_update_camera_info,  # Extract and update camera information
    update_camera_data_dataframe,    # Update camera data DataFrame

    ### Functions to detect and classify animals in photos ###

    set_device,                 # Sets computation device (CPU/GPU)
    detector,                   # Animal/human/vehicle detection
    classifier,                 # Animal classification model
    batch_classification,       # Batch classification of images
    detections_in_sequence,     # Checks if anything has been detected

    ### Functions to annotate photos and send an alert to Telegram ###

    generate_alert_caption,     # Generate captions for alerts
    send_alert_to_telegram,     # Send alerts to Telegram
    annotate_images,            # Annotate images with detection results

    ### Functions to save the photos and send weekly reports ###
    save_images,                # Save images to disk
    send_weekly_report          # Send a weekly report
)

### Step 2: Test Your Setup on Example Photos




In [ ]:
# Detection Model Settings
DETECTOR_MODEL_PATH = '../models/md_v5a.0.0.pt'
DETECTOR_CLASSES = ["animal", "human", "vehicle"]
DETECTION_THRESHOLD = 0.15      # Detection threshold - recommended 0.15

# Classification Model Settings
BACKBONE = 'vit_large_patch14_dinov2'
CLASSIFIER_MODEL_PATH = '../models/deepfaune-vit_large_patch14_dinov2.lvd142m.pt'   # Change to fine-tuned model if desired
CLASSIFIER_CLASSES = [
    "Badger", "Ibex", "Red Deer", "Chamois", "Cat",
    "Goat", "Roe Deer", "Dog", "Squirrel", "Equid", "Genet",
    "Hedgehog", "Lagomorph", "Wolf", "Lynx", "Marmot",
    "Micromammal", "Mouflon", "Sheep", "Mustelid", "Bird",
    "Bear", "Nutria", "Fox", "Wild Boar", "Cow"
]
ROMANIAN_CLASSES = [
    "Bursuc", "Ibex", "Cerb", "Capră Neagră", "Pisică", 
    "Capră", "Căprioară", "Câine", "Veveriță", "Cal", "Genetă",
    "Arici", "Iepuri", "Lup", "Râs", "Marmotă", 
    "Micromamifer", "Muflon", "Oaie", "Mustelid", "Pasăre", 
    "Urs", "Nutrie", "Vulpe", "Mistreț", "Vacă"
]
SPECIES_OF_INTEREST = ["Wild Boar", "Bear"] # Species for which priority alerts are sent (currenly only supports wild boar and bears)
CLASSIFICATION_THRESHOLD = 0.20 # Classification threshold - recommend set between 0.2 to 0.8 depending on tolerance for false positives

# Alert Message Settings
ALERT_LANGUAGE = "en"           # Curently English (en) and Romanian (ro) are supported
HUMAN_ALERT_START = "21:00"     # Privacy feature: start/end time that photos of people may be sent
HUMAN_ALERT_END = "06:00"

CAPTURE_DATABASE_PATH = '../data/capture_database.csv' 
CAMERA_LOCATIONS_PATH = '../data/camera_locations.csv'
PHOTOS_PATH = '../data/photos/' # Storage folder for photos received

# Initialise the Detection and Classifier Models
device = set_device()
detector_model = run_detector.load_detector(DETECTOR_MODEL_PATH)
print("Loading classifier...")
start_time = time.time()
classifier_model = classifier(CLASSIFIER_MODEL_PATH, BACKBONE, CLASSIFIER_CLASSES, device)
end_time = time.time()
print(f"Loaded classifier in {(end_time - start_time):.2f} seconds")

In [ ]:
# Function to open file dialog and select one or more JPG images
def open_images():
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    root.geometry("800x600")  
    initial_dir = '../data/example_photos'  # Set initial directory for file dialog
    # Open file dialog to select one or more JPG files
    file_paths = filedialog.askopenfilenames(initialdir=initial_dir, title="Select file(s)",
                                             filetypes=[("JPEG files", "*.jpg"), 
                                                        ("JPEG files", "*.jpeg"), 
                                                        ("JPEG files", "*.JPG"), 
                                                        ("JPEG files", "*.JPEG")])
    # Open and return the selected images as a list of PIL Image objects
    return [Image.open(file_path) for file_path in file_paths] if file_paths else []


# Call the function and store the selected images in a list
images = open_images()

# Print the number of selected images
print(f"{current_time()} | {len(images)} image(s) selected")

# Load the capture database into a DataFrame, df
# This DataFrame is required to manage data in the subsequent steps
df = pd.read_csv(CAPTURE_DATABASE_PATH)

# Loop to add dummy rows to the DataFrame for this example
for _ in range(len(images)):
    new_row = pd.DataFrame([['Example', 1] + ['Example'] * 10 + [None] * (len(df.columns) - 12)], columns=df.columns)
    df = pd.concat([df, new_row], ignore_index=True)

# Detect animals, humans and/or vehicles in the image(s) using the MegaDetector detection model
df, human_warning = detector(df, detector_model, images, DETECTION_THRESHOLD)

# Classify objects in the image(s) using the DeepFaune classification model
df = batch_classification(df, classifier_model, images, CLASSIFICATION_THRESHOLD)

# Check if there are any detections in the sequence above the threshold
if detections_in_sequence(df, images):
    # Generate an alert caption based on the detections
    df, alert_caption = generate_alert_caption(df, human_warning, HUMAN_ALERT_START, HUMAN_ALERT_END, len(images), SPECIES_OF_INTEREST, "example@email.com", ALERT_LANGUAGE, CLASSIFIER_CLASSES, ROMANIAN_CLASSES)
    human_warning = False  # Reset human warning flag
    # Annotate images with bounding boxes, labels and confidence levels
    images = annotate_images(df, images, human_warning, HUMAN_ALERT_START, HUMAN_ALERT_END, ALERT_LANGUAGE, CLASSIFIER_CLASSES, ROMANIAN_CLASSES)

# Display the images in Jupyter notebook
if images:
    html = "<div style='display: flex; flex-wrap: wrap;'>"
    for img in images:
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        html += f"<div style='margin: 10px;'><img src='data:image/jpeg;base64,{img_str}' width='600px' /></div>"
    html += "</div>"
    display(HTML(html))

# Display the alert message in Jupyter notebook
alert_html = f"<div style='padding: 10px; background-color: white; border: 1px solid black; white-space: pre-wrap; color: black; width: 600px;'>{alert_caption}</div>"
display(HTML(alert_html))


### Step 3: Set Up an Alert System Email Account and Telegram Group

- Gmail example
- Telegram example
- Config file structure


In [ ]:

# Load settings from configuration file
with open('../config.yaml') as file:
    config = yaml.safe_load(file)

IMAP_HOST = config['imap_config']['host']
EMAIL_USER = config['imap_config']['user']
EMAIL_PASS = config['imap_config']['password']

TELEGRAM_BOT_TOKEN = config['telegram_config']['bot_token']
TELEGRAM_CHAT_ID =  '-1002249589791' # config['telegram_config']['chat_id']  #  replace with config after tests # 

SMTP_SERVER = config['smtp_config']['host']
EMAIL_SENDER = config['smtp_config']['user']
EMAIL_PASSWORD = config['smtp_config']['password']
RECIPIENTS = config['smtp_config']['recipients']

SMTP_PORT = 587
CHECK_EMAIL_FREQUENCY = 10

schedule.every().monday.at("08:00").do(
    functools.partial(send_weekly_report, SMTP_SERVER, EMAIL_SENDER, EMAIL_PASSWORD, SMTP_PORT, CAPTURE_DATABASE_PATH, CAMERA_LOCATIONS_PATH, RECIPIENTS, EMAIL_USER)
)


### Step 4: Verify Your Setup

In [ ]:

if __name__ == "__main__":
    print(f"\n{current_time()} | Monitoring {EMAIL_USER} for new messages...")
    while True:
        try:
            images, original_images, camera_id, temp_deg_c, img_date, img_time, battery, sd_memory = \
                check_emails(IMAP_HOST, EMAIL_USER, EMAIL_PASS)
            if camera_id:
                camera_make, gps, location, map_url, battery, sd_memory = extract_and_update_camera_info(CAMERA_LOCATIONS_PATH, camera_id, battery, sd_memory)
                if images:
                    df = pd.read_csv(CAPTURE_DATABASE_PATH)
                    df = update_camera_data_dataframe(df, len(images), camera_id, camera_make, img_date, img_time, temp_deg_c, battery, sd_memory, location, gps, map_url)
                    df, human_warning = detector(df, detector_model, images, DETECTION_THRESHOLD)
                    df = batch_classification(df, classifier_model, images, CLASSIFICATION_THRESHOLD)
                    if detections_in_sequence(df, images):
                        df, alert_caption = generate_alert_caption(df, human_warning, HUMAN_ALERT_START, HUMAN_ALERT_END, len(images), SPECIES_OF_INTEREST, EMAIL_USER, ALERT_LANGUAGE, CLASSIFIER_CLASSES, ROMANIAN_CLASSES)
                        
                        ### DELETE ME ####
                        human_warning = False
                        
                        alert_images = annotate_images(df, images, human_warning, HUMAN_ALERT_START, HUMAN_ALERT_END, ALERT_LANGUAGE, CLASSIFIER_CLASSES, ROMANIAN_CLASSES)

                        send_alert_to_telegram(TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, alert_images, alert_caption)
                    else:
                        print(f"{current_time()} | All photos in sequence are empty")
                    df = save_images(df, original_images, human_warning, PHOTOS_PATH)
                    df.to_csv(CAPTURE_DATABASE_PATH, index=False)
                    print(f"{current_time()} | Capture database updated: {CAPTURE_DATABASE_PATH}")
                    del df
                else:
                    print(f"{current_time()} | No images attached to email")
                print(f"\n{current_time()} | Monitoring {EMAIL_USER} for new messages...")
            else:
                time.sleep(CHECK_EMAIL_FREQUENCY)
            schedule.run_pending()
        except KeyboardInterrupt:
            print(f"{current_time()} | Interrupted by user")
            break

        except Exception as e:
            print(f"{current_time()} | An error occurred: {e}")
            time.sleep(CHECK_EMAIL_FREQUENCY)
            print(f"\n{current_time()} | Monitoring {EMAIL_USER} for new messages...")
            continue

### Step 5: Deploy Your Script